In [ ]:
def calculate_by_metric(var, metric):
    if metric == "root_mean_sq_diff":
        return null
    elif metric == "global_mean_anomaly_control":
        exp='G6sulfur'
        project='GeoMIP'
        after=['2070-01-01','2100-01-01'] #with geoengineering
        before=['1960-01-01','1990-01-01'] #control (pre-geoengineering/warming)
        control_mean=global_mean(var,exp,before,project)
        geo_mean=global_mean(var,exp,after,project)
        return geo_mean - control_mean   
    elif metric == "global_mean_anomaly_solar":
        dates=['2070-01-01','2100-01-01']
        sulfur='G6sulfur'
        solar='G6solar'
        project='GeoMIP'
        solar_mean=global_mean(var,solar,dates,project)
        sulfur_mean=global_mean(var,sulfur,dates,project)
        return sulfur_mean - solar_mean
    elif metric == "global_mean_anomaly_warming":
        geo_exp='G6sulfur'
        warming_exp="rcp45-cmip5"
        geo_project='GeoMIP'
        warming_project="ScenarioMIP"
        dates=['2070-01-01','2100-01-01']
        warming_mean=global_mean(var,warming_exp,dates,warming_project)
        geo_mean=global_mean(var,geo_exp,after,geo_project)
        return geo_mean - warming_mean
    elif metric == "moderated_exacerbated":
        geo_exp='G6sulfur'
        warming_exp="rcp45-cmip5"
        geo_project='GeoMIP'
        warming_project="ScenarioMIP"
        dates=['2070-01-01','2100-01-01']
        warming_mean=global_mean(var,warming_exp,dates,warming_project)
        geo_mean=global_mean(var,geo_exp,after,geo_project)
        before=['1960-01-01','1990-01-01'] #control (pre-geoengineering/warming)
        control_mean=global_mean(var,geo_exp,before,geo_project)
        geo_anom = geo_mean - control_mean
        warming_anom = warming_mean - control_mean
        if geo_anom < warming_anom:
            return "moderated"
        elif warming_anom < geo_anom:
            return "exacerbated"
        else:
            return "N/A"
    else:
        return null

In [1]:
#Only works for one run at a time. Multi-run code is in quigley_code.ipynb

def global_mean(var,exp,dates,project):
    data_dir='/home/users/quigley/Projects/geoengineering_summer' #Changed from Pete's directory
    model='UKESM1-0-LL'
    centre='MOHC'
    domain='Amon'
    run='r1i1p1f2'
    grid='gn'
    season='ANN'
    time_files=1
    
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')

    # load the arguments into a list then... 
    args=[season,dates,data_dir,model,centre,var,domain,exp,project,run,grid,time_files]
    ds_mean, ds_std = get_seasonal_mean_std(*args) # ... unpack them into the function.
    
    # multiply each temperature value with its corresponding fraction of total area, then sum to find the global mean.
    return (ds_mean[var]*da_weight).sum() - 273.15 # convert from K to C.